In [10]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""This notebook updates de SCI database from the content of the annexes.
It does not recreate the content of the database from the annexes, as the
initial version was created from the original databases."""

import os, sys, inspect

try :
    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    import mwparserfromhell as mwh

from io import StringIO
import pandas as pd
import numpy as np

In [11]:
TAG                 = 'WLE'

BASE_SITE_DB_NAME   = "Commons:Wiki Loves Earth 2016 in Spain"
SITE_DB_PAGE        = BASE_SITE_DB_NAME + "/SCI DB"

commons_site = pb.Site("commons", "commons")
wikipedia_site = pb.Site("es", "wikipedia")

In [12]:
annexes = {
   'ES-AN': [u'Anexo:Lugares de importancia comunitaria de Andalucía', 'Andalusia'],
   'ES-AR': [u'Anexo:Lugares de importancia comunitaria de Aragón', 'Aragon'],
   'ES-AS': [u'Anexo:Lugares de importancia comunitaria de Asturias', 'Asturias'],
   'ES-CB': [u'Anexo:Lugares de importancia comunitaria de Cantabria', 'Cantabria'],
   'ES-CM': [u'Anexo:Lugares de importancia comunitaria de Castilla-La Mancha', 'Castile-La Mancha'],
   'ES-CL': [u'Anexo:Lugares de importancia comunitaria de Castilla y León', u'Castile and León'],
   'ES-CT': [u'Anexo:Lugares de importancia comunitaria de Cataluña', 'Catalonia'],
   'ES-MD': [u'Anexo:Lugares de importancia comunitaria de la Comunidad de Madrid', 'Community of Madrid'],
   'ES-VC': [u'Anexo:Lugares de importancia comunitaria de la Comunidad Valenciana', 'Valencian Community'],
   'ES-EX': [u'Anexo:Lugares de importancia comunitaria de Extremadura', 'Extremadura'],
   'ES-IB': [u'Anexo:Lugares de importancia comunitaria de las Islas Baleares', 'Balearic Islands'],
   'ES-CN': [u'Anexo:Lugares de importancia comunitaria de las Islas Canarias', 'Canary Islands'],
   'ES-GA': [u'Anexo:Lugares de importancia comunitaria de Galicia', 'Galicia'],
   'ES-RI': [u'Anexo:Lugares de importancia comunitaria de La Rioja', 'La Rioja'],
   'ES-NC': [u'Anexo:Lugares de importancia comunitaria de Navarra', 'Navarre'],
   'ES-MC': [u'Anexo:Lugares de importancia comunitaria de la Región de Murcia', 'Region of Murcia'],
   'ES-PV': [u'Anexo:Lugares de importancia comunitaria del País Vasco', 'Basque Country'],
   'ES-ML': [u'Anexo:Lugares de importancia comunitaria de Ceuta y Melilla', 'Melilla'],
   'ES-CE': [u'Anexo:Lugares de importancia comunitaria de Ceuta y Melilla', 'Ceuta'],
   'ES-MAGRAMA': [u'Anexo:Lugares de importancia comunitaria del MAGRAMA', 'MAGRAMA']
}
annexes_list = list(set([annexes[key][0] for key in annexes]))

## Auxiliary functions

In [13]:
def format_row(x) :
    """This function is used to prevent the transformation of some columns to floats when saving as CSV"""
    try:
        return int(x)
    except :
        return ''

## Annex information retrieval

In [14]:
sites_df = pd.DataFrame(
    columns=['lic_id', 'commons_cat', 'image'])

for annex in annexes_list:
    pb.output("Processing annex " + annex)
    page = pb.Page(wikipedia_site, annex)
    text = page.text
    wikicode = mwh.parse(text)
    templates = [template for template in wikicode.filter_templates() if template.name.lower().strip() == "fila lic"]
    for template in templates :
        df_row = {
            "lic_id": template.get("código").value.strip(),
            "commons_cat": '',
            "image": ''
        }
        try:
            if template.get("categoría-Commons").value :
                df_row["commons_cat"] = template.get("categoría-Commons").value.strip()
            if template.get("imagen").value :
                df_row["image"] = template.get("imagen").value.strip()
        except :
            print (template.get("código").value.strip())

        sites_df = sites_df.append(df_row, ignore_index=True)

Processing annex Anexo:Lugares de importancia comunitaria de Andalucía
Processing annex Anexo:Lugares de importancia comunitaria del MAGRAMA
Processing annex Anexo:Lugares de importancia comunitaria de Navarra
Processing annex Anexo:Lugares de importancia comunitaria de Cantabria
Processing annex Anexo:Lugares de importancia comunitaria de las Islas Baleares
Processing annex Anexo:Lugares de importancia comunitaria de las Islas Canarias
Processing annex Anexo:Lugares de importancia comunitaria de la Región de Murcia
Processing annex Anexo:Lugares de importancia comunitaria del País Vasco
Processing annex Anexo:Lugares de importancia comunitaria de Ceuta y Melilla
Processing annex Anexo:Lugares de importancia comunitaria de Asturias
Processing annex Anexo:Lugares de importancia comunitaria de Extremadura
Processing annex Anexo:Lugares de importancia comunitaria de Aragón
Processing annex Anexo:Lugares de importancia comunitaria de Castilla y León
Processing annex Anexo:Lugares de import

## Retrieval of exising database

In [15]:
pb.output('Retrieving --> WLE SCI list')
site_list_page = pb.Page(commons_site, SITE_DB_PAGE)
site_list_text = StringIO(site_list_page.text[site_list_page.text.find('\n') + 1:site_list_page.text.rfind('\n')])
site_df = pd.read_csv(site_list_text, sep=";",
                     index_col=False,
                     names=["name", "lic_id", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"],
                     dtype={"latitude": float, "longitude": float, "area": float, 
                            "marine_percentage": float, "marine_area": float}
                     )
#site_df.fillna('')
pb.output('Retrieved --> WLE SCI list')

Retrieving --> WLE SCI list
Retrieved --> WLE SCI list


## Merging

In [16]:
site_df.drop(['commons_cat', 'image'], axis=1, inplace=True)

In [17]:
columns = ["name", "lic_id", "magrama_url", "community",
           "bio_region", "continent", "min_altitude",
           "max_altitude", "avg_altitude", "longitude",
           "latitude", "area", "marine_percentage",
           "marine_area", "image", "commons_cat", "wikidata_id"]

result = pd.merge(site_df, sites_df, on='lic_id', how='outer')
result['longitude'] = result['longitude'].round(6)
result['latitude'] = result['latitude'].round(6)
result['area'] = result['area'].round(3)
result['max_altitude'] = result['max_altitude'].apply(lambda x: format_row(x))
result['min_altitude'] = result['min_altitude'].apply(lambda x: format_row(x))
result['avg_altitude'] = result['avg_altitude'].apply(lambda x: format_row(x))
result = result[columns]

## Updated database upload

In [18]:
buf = StringIO()
result.to_csv(buf, index=None, sep=';', encoding='utf-8', header=False)

sites_page = pb.Page(commons_site, SITE_DB_PAGE)
sites_page.text = '<pre>\n' + buf.getvalue() + '</pre>'
sites_page.save("{} in Spain: Sites database update".format(TAG))

Page [[commons:Commons:Wiki Loves Earth 2016 in Spain/SCI DB]] saved
